In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
#from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Embedding
from google.colab import files
#adicionando novas bibliotecas
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [ ]:
files.upload()

In [ ]:
spam = pd.read_csv("spam.csv")
spam.head()

In [5]:
#Encoder para as classes do dataset (spam - not-spam)
labelencoder = LabelEncoder()
y = labelencoder.fit_transform(spam["Category"])
print(y)

[0 0 1 ... 0 0 0]


In [6]:
mensagens = spam["Message"].values
x_train, x_test, y_train, y_test = train_test_split(mensagens, y, test_size=0.3)

In [ ]:
#------------Remover------------
#Vetorizando os documentos
#vetorizador = CountVectorizer()
#vetorizador.fit(x_train)
#x_train = vetorizador.transform(x_train)
#x_test = vetorizador.transform(x_test)

In [ ]:
#Tokenização
token = Tokenizer(num_words=1000)
token.fit_on_texts(x_train)

x_train = token.texts_to_sequences(x_train)
x_test = token.texts_to_sequences(x_test)

print(x_train)

In [ ]:
x_train = pad_sequences(x_train, padding='post', maxlen=500)
x_test = pad_sequences(x_test, padding='post', maxlen=500)
print(x_train)

In [15]:
#criando a rede profunda
modelo = Sequential()
modelo.add(Embedding(input_dim=len(token.word_index), output_dim=50, input_length=500))
modelo.add(Flatten())
modelo.add(Dense(units=10, activation="relu"))
modelo.add(Dropout(0.1))
modelo.add(Dense(units=1, activation="sigmoid"))

In [16]:
#Compilando o modelo, passando parâmetros loss, optimizer e metrics
modelo.compile(loss="mean_squared_error", optimizer="adam", metrics=["accuracy"])

In [17]:
#Visualizando o modelo
modelo.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 500, 50)           378200    
                                                                 
 flatten_1 (Flatten)         (None, 25000)             0         
                                                                 
 dense_3 (Dense)             (None, 10)                250010    
                                                                 
 dropout_2 (Dropout)         (None, 10)                0         
                                                                 
 dense_4 (Dense)             (None, 1)                 11        
                                                                 
Total params: 628221 (2.40 MB)
Trainable params: 628221 (2.40 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [22]:
#treinamento da rede
modelo.fit(x_train, y_train, epochs=30, verbose=True, batch_size=10,
           validation_data=(x_test, y_test))

Epoch 1/30
390/390 [==============================] - 8s 20ms/step - loss: 0.0069 - accuracy: 0.9864 - val_loss: 0.0120 - val_accuracy: 0.9868
Epoch 2/30
390/390 [==============================] - 7s 18ms/step - loss: 0.0063 - accuracy: 0.9897 - val_loss: 0.0110 - val_accuracy: 0.9886
Epoch 3/30
390/390 [==============================] - 6s 14ms/step - loss: 0.0056 - accuracy: 0.9938 - val_loss: 0.0120 - val_accuracy: 0.9874
Epoch 4/30
390/390 [==============================] - 7s 18ms/step - loss: 0.0058 - accuracy: 0.9954 - val_loss: 0.0108 - val_accuracy: 0.9892
Epoch 5/30
390/390 [==============================] - 6s 15ms/step - loss: 0.0051 - accuracy: 0.9954 - val_loss: 0.0110 - val_accuracy: 0.9880
Epoch 6/30
390/390 [==============================] - 6s 17ms/step - loss: 0.0045 - accuracy: 0.9962 - val_loss: 0.0110 - val_accuracy: 0.9886
Epoch 7/30
390/390 [==============================] - 6s 15ms/step - loss: 0.0057 - accuracy: 0.9954 - val_loss: 0.0114 - val_accuracy: 0.9874

In [23]:
#Matriz de confusão
previsao = modelo.predict(x_test)

53/53 [==============================] - 0s 5ms/step


In [24]:
print(previsao)

[[2.2603651e-06]
 [7.0313894e-11]
 [8.9349567e-07]
 ...
 [8.4778442e-09]
 [1.0000000e+00]
 [5.3008384e-09]]


In [25]:
previsao_convertido = (previsao > 0.5)
matriz_confusao = confusion_matrix(y_test, previsao_convertido)
print(matriz_confusao)

[[1452    9]
 [  15  196]]
